# Tool Level HITL Control

You can use this to add HITL to any tool without having to add it inside the tool!

### Define basic stuff

In [ ]:
import requests
import yfinance as yf
from pprint import pformat
from langchain_core.tools import tool


########################### Tools 
@tool("lookup_stock")
def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


@tool("fetch_stock_data")
def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}


@tool
def place_order(
    symbol: str,
    action: str,
    shares: int,
    limit_price: float,
    order_type: str = "limit",
) -> dict:
    """
    Execute a stock order.

    Parameters:
    - symbol: Ticker
    - action: "buy" or "sell"
    - shares: Number of shares to trade (pre-computed by the agent)
    - limit_price: Limit price per share
    - order_type: Order type, default "limit"

    Returns:
    - status: Execution result (simulated)
    - symbol
    - shares
    - limit_price
    - total_spent
    - type: Order type used
    - action
    """
    total_spent = round(int(shares) * limit_price, 2)
    return {
        "status": "filled",
        "symbol": symbol,
        "shares": int(shares),
        "limit_price": limit_price,
        "total_spent": total_spent,
        "type": order_type,
        "action": action,
    }


########################### Prompt
system_message = """
You are a financial advisor assistant. Use the provided tools to ground your answers
in up-to-date market data. Be concise, factual, and risk-aware.

Be decisive: when you have sufficient information to act, proceed with tool calls without
asking for confirmation. Only if information is missing or uncertain, ask a concise 
clarifying question.

When preparing or describing actions, include appropriate parameters (e.g., symbol, shares,
limit price, budgets) based on available data. Do not fabricate numbers or facts.
"""


########################### pretty printing
def print_tool_approval(payload):
    tool = payload.get("awaiting", "unknown_tool")
    args = payload.get("args", {})

    print("—-- Approval needed —--")
    print(f"Tool: {tool}")

    if isinstance(args, dict) and args:
        print("Parameters:")
        for k, v in args.items():
            print(f"  - {k}: {v}")
    elif args:
        print(f"Parameters: {args}")
    else:
        print("No parameters.")

## Use post model hook for Risky Tools (previous experience from lesson 4)

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from tools import draw_mermaid_png
from langchain_core.messages import AIMessage
from langgraph.types import interrupt

RISKY_TOOLS = {"place_order"}

def halt_on_risky_tools(state):
    last = state["messages"][-1]
    if isinstance(last, AIMessage) and getattr(last, "tool_calls", None):
        for tc in last.tool_calls:
            if tc.get("name") in RISKY_TOOLS:
                decision = interrupt({"awaiting": tc["name"], "args": tc.get("args", {})})

                # tool approved
                if isinstance(decision, dict) and decision.get("approved"):
                    return {}

                # tool rejected
                tool_msg = ToolMessage(
                    content=f"Cancelled by human. Continue without executing that tool and provide next steps.",
                    tool_call_id=tc["id"],
                    name=tc["name"]
                )
                return {"messages": [tool_msg]}

    return {}

agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[lookup_stock_symbol, fetch_stock_data_raw, place_order],
    prompt=system_message,
    checkpointer=InMemorySaver(),

    version="v2",
    post_model_hook=halt_on_risky_tools
)

draw_mermaid_png(agent)

## Introduce a universal tool wrapper for HITL (you can wrap any tool you need!)

### create wrapper

In [ ]:
from typing import Callable
from langchain_core.tools import BaseTool, tool
from langchain_core.runnables import RunnableConfig
from langgraph.types import interrupt

def add_approval(main_tool: Callable | BaseTool) -> BaseTool:
    """Wrap a tool to support human-in-the-loop review."""
    if not isinstance(main_tool, BaseTool):
        main_tool = tool(main_tool)

    @tool(  
        main_tool.name,
        description=main_tool.description,
        args_schema=main_tool.args_schema
    )
    def call_main_tool_with_hitl(config: RunnableConfig, **tool_input):
        decision = interrupt({
            "awaiting": main_tool.name,
            "args": tool_input
        })

        # tool approved
        if isinstance(decision, dict) and decision.get("approved"):
            return main_tool.invoke(tool_input, config)

        # tool rejected
        return "Cancelled by human. Continue without executing that tool and provide next steps."
        

    return call_main_tool_with_hitl

## prepare list of available tools

In [ ]:
RISKY_TOOLS = {"place_order"}

tools = [lookup_stock_symbol, fetch_stock_data_raw, place_order]

# Wrap risky tools with approval mechanism
tools = [
    add_approval(tool) if tool.name in RISKY_TOOLS else tool
    for tool in tools
]

### create ReAct agent

In [ ]:
agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=tools,
    prompt=system_message,
    checkpointer=InMemorySaver(),
)

draw_mermaid_png(agent)

## Testing

### Approval

In [ ]:
from langchain_core.messages import HumanMessage
import uuid

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4())
    }
}

response = agent.invoke({"messages": [HumanMessage(content="Buy $1000 of Tesla stock at the current price.")]}, config)
for message in response['messages']:
    message.pretty_print()

In [ ]:
interrupts = response["__interrupt__"]
print_tool_approval(interrupts[0].value)

In [ ]:
from langgraph.types import Command

response = agent.invoke(Command(resume={"approved": True}), config=config)
for message in response['messages']:
    message.pretty_print()

In [ ]:
"__interrupt__" in response

### Cancel

In [ ]:
from langchain_core.messages import HumanMessage
import uuid

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4())
    }
}

response = agent.invoke({"messages": [HumanMessage(content="Buy $1000 of Tesla stock at the current price.")]}, config)
for message in response['messages']:
    message.pretty_print()

In [ ]:
interrupts = response["__interrupt__"]
print_tool_approval(interrupts[0].value)

In [ ]:
response = agent.invoke(Command(resume={"approved": False}), config=config)
for message in response['messages']:
    message.pretty_print()

In [ ]:
"__interrupt__" in response

## Reference Links

**1. Human-in-the-Loop: Add Interrupts to Any Tool**

https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/add-human-in-the-loop/#add-interrupts-to-any-tool

→ Guide to wrapping tools with human-in-the-loop interrupts, enabling approval workflows for specific tool calls before execution, with patterns for tool interception and user confirmation.